# Case de Entrevista – Cientista de Dados (Welhome)

**Autor:** Gabriel Fabiano de Souza

**Data:** 30/09/2025

Para updates e novidades: https://github.com/edr0k/chatbot-welhome

In [18]:
# !pip install --upgrade langchain langchain-groq langchain-huggingface faiss-cpu python-dotenv sentence-transformers pydantic

import os
from dotenv import load_dotenv

# Importações para Groq (LLM) e Google (Embeddings)
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Importações do LangChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Importações para parsing estruturado e validação
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# Carrega a GROQ_API_KEY e a GOOGLE_API_KEY do seu arquivo .env
load_dotenv()
print("Bibliotecas importadas e .env carregado.")

# Inicializa o LLM da Groq
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.7)

# Inicializa o modelo de embeddings da Google Generative AI
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

print("Modelo LLM (Groq Llama 3) e Embeddings (Google Generative AI) inicializados.")

Bibliotecas importadas e .env carregado.
Modelo LLM (Groq Llama 3) e Embeddings (Google Generative AI) inicializados.


## PARTE 1 - CHATBOT ASSISTENTE COM MEMÓRIA

In [19]:
# Prompt que define o comportamento do assistente
prompt_chat = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente virtual da Welhome, amigável e eficiente. Seu objetivo é fazer uma qualificação inicial do lead, fazendo as perguntas necessárias uma de cada vez. Seja cordial e natural."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# Chain de conversação com a sintaxe moderna do LangChain (LCEL)
runnable = prompt_chat | llm

# Armazenamento em memória para o histórico das conversas
# Cada lead terá seu histórico separado por um 'session_id'
store = {}
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrapper que adiciona a gestão de memória à nossa chain
conversation_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("Chatbot com memória configurado.")

Chatbot com memória configurado.


In [20]:
print("--- Iniciando Simulação da Conversa ---")
session_id = "lead_teste_001"
config = {"configurable": {"session_id": session_id}}

# Simula uma conversa completa para coletar os dados
print("Assistente:", conversation_with_history.invoke({"input": "Olá, eu gostaria de saber mais sobre os serviços da Welhome."}, config=config).content)
print("Assistente:", conversation_with_history.invoke({"input": "Claro! Eu tenho 3 imóveis em São Paulo e estou cansado de administrar tudo sozinho."}, config=config).content)
print("Assistente:", conversation_with_history.invoke({"input": "Minha experiência é bem básica, só anoto as coisas numa planilha e sempre me perco."}, config=config).content)
print("--- Simulação Concluída ---")

--- Iniciando Simulação da Conversa ---
Assistente: Olá! É um prazer conversar com você! A Welhome é uma empresa especializada em serviços de gestão de propriedades e soluções para proprietários e inquilinos. Nossa equipe está aqui para ajudar a tornar a experiência de morar ou alugar uma propriedade mais fácil e agradável.

Antes de começarmos a conversar sobre os serviços específicos, gostaria de saber: você é proprietário de uma propriedade ou está procurando por uma solução para alugar um imóvel?
Assistente: Entendo perfeitamente! Administrar múltiplas propriedades pode ser um desafio, especialmente em uma cidade grande como São Paulo. A Welhome pode ajudar a aliviar essa carga e fornecer soluções personalizadas para gerenciar seus imóveis de forma eficiente.

Você mencionou que está cansado de administrar tudo sozinho, então posso imaginar que você está procurando por uma solução que possa ajudar a reduzir o estresse e o tempo gasto com a gestão dos imóveis.

Pode me dizer, quais 

In [21]:
# Pega o histórico da conversa que acabamos de simular
historico_conversa = get_session_history(session_id).messages

# --- Definição das Estruturas de Saída com Pydantic ---
# Isso garante que a saída do LLM será sempre um JSON estruturado e validado.
class LeadAnalysis(BaseModel):
    score: int = Field(description="Uma pontuação de 1 a 10 indicando o potencial do lead, onde 10 é muito alto.")
    intent: str = Field(description="A intenção do lead, classificada como 'Pronto para contratar', 'Buscando informações' ou 'Apenas curioso'.")
    reasoning: str = Field(description="Uma breve explicação em uma frase sobre o porquê da pontuação e da intenção atribuídas.")

# --- Criação do Parser e do Prompt ---
output_parser = PydanticOutputParser(pydantic_object=LeadAnalysis)

prompt_analise = ChatPromptTemplate.from_template("""
Analise o histórico de conversa com um potencial cliente da Welhome.
Com base na conversa, atribua uma pontuação de potencial e classifique a intenção do lead.
Considere fatores como número de imóveis, "dores" mencionadas (ex: 'cansado de administrar') e nível de experiência.
Um lead com mais imóveis e mais "dores" tem um score maior.

{format_instructions}

Histórico da Conversa:
{historico}
""")

# --- Criação e Execução da Chain de Análise ---
analysis_chain = prompt_analise | llm | output_parser
lead_analysis_result = analysis_chain.invoke({
    "historico": historico_conversa,
    "format_instructions": output_parser.get_format_instructions()
})

print("--- Análise Avançada do Lead ---")
print(f"Pontuação do Lead: {lead_analysis_result.score}/10")
print(f"Intenção Detectada: {lead_analysis_result.intent}")
print(f"Justificativa: {lead_analysis_result.reasoning}")

--- Análise Avançada do Lead ---
Pontuação do Lead: 8/10
Intenção Detectada: Pronto para contratar
Justificativa: O lead tem 3 imóveis e está procurando por uma solução para aliviar a carga de trabalho e reduzir o estresse, indicando um alto potencial de contratação.


In [31]:
class LeadSummary(BaseModel):
    nome: str = Field(description="Nome do lead, se mencionado. Caso contrário, 'Não informado'.")
    quantidade_imoveis: int = Field(description="Número de imóveis que o lead possui.")
    localizacao: str = Field(description="Cidade ou região dos imóveis.")
    experiencia_previa: str = Field(description="Descrição da experiência do lead com administração de imóveis.")
    lead_score: int = Field(description="A pontuação de potencial do lead (1-10).")
    intencao: str = Field(description="A intenção classificada do lead.")
    resumo_qualitativo: str = Field(description="Um resumo de 2-3 frases sobre a principal necessidade e situação do lead.")

summary_parser = PydanticOutputParser(pydantic_object=LeadSummary)

prompt_resumo_final = ChatPromptTemplate.from_template("""
Extraia as informações do histórico de conversa e da análise do lead para criar um resumo estruturado para o vendedor.

{format_instructions}

Histórico da Conversa:
{historico}

Análise de Score e Intenção:
- Score: {score}
- Intenção: {intencao}
""")

summary_chain = prompt_resumo_final | llm | summary_parser

final_summary = summary_chain.invoke({
    "historico": historico_conversa,
    "score": lead_analysis_result.score,
    "intencao": lead_analysis_result.intent,
    "format_instructions": summary_parser.get_format_instructions()
})

print("--- Resumo Estruturado para o Vendedor ---")
for item, value in final_summary.model_dump().items():
    print(f"{item.replace('_', ' ').capitalize()}: {value}")

--- Resumo Estruturado para o Vendedor ---
Nome: Não informado
Quantidade imoveis: 3
Localizacao: São Paulo
Experiencia previa: Básica, utilizando planilha para anotar informações
Lead score: 8
Intencao: Pronto para contratar
Resumo qualitativo: O lead possui 3 imóveis em São Paulo e está procurando por uma solução para gerenciar seus imóveis de forma eficiente, pois está cansado de administrar tudo sozinho. Ele tem experiência básica em administração de imóveis e está pronto para contratar serviços de gestão de propriedades.


## PARTE 2 - SISTEMA DE RAG COM FAQ

In [ ]:
# FAQ da Welhome
faq_welhome = """
Pergunta: Como a Welhome garante que o aluguel será pago em dia?
Resposta: A Welhome oferece a garantia de aluguel em dia. Mesmo que o inquilino atrase, nós garantimos o repasse do valor para o proprietário na data combinada, sem custos adicionais.

Pergunta: Quem é responsável pela manutenção do imóvel?
Resposta: Reparos estruturais são de responsabilidade do proprietário. Pequenas manutenções do dia a dia, como troca de lâmpadas, são do inquilino. Nossa plataforma ajuda a intermediar e orçar serviços de manutenção quando necessário.

Pergunta: Qual é o custo da taxa de administração da Welhome?
Resposta: Nossa taxa de administração é de 8% sobre o valor do aluguel. Essa taxa cobre a gestão completa do seu imóvel, incluindo divulgação, gestão de contratos, repasse garantido e suporte.

Pergunta: Como funciona a vistoria do imóvel?
Resposta: Realizamos uma vistoria profissional completa, com fotos e vídeos, antes da entrada do inquilino e após a sua saída. Isso garante que o imóvel seja devolvido nas mesmas condições em que foi entregue.
"""

In [ ]:
# Dividir e Vetorizar
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([faq_welhome])
vector_store = FAISS.from_documents(docs, embeddings)
retriever = vector_store.as_retriever()

# Prompt e Chain de RAG
prompt_rag_template = ChatPromptTemplate.from_template("""
Responda a pergunta do usuário de forma natural com base apenas no contexto fornecido:
Contexto:
{context}

Pergunta: {input}
""")

question_answer_chain = create_stuff_documents_chain(llm, prompt_rag_template)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("Setup do RAG concluído.")


Setup do RAG concluído.


In [ ]:
pergunta_teste_rag = "Quanto vocês cobram pelo serviço?"
resposta_rag = rag_chain.invoke({"input": pergunta_teste_rag})

print(f"\nPERGUNTA: {pergunta_teste_rag}")
print(f"RESPOSTA: {resposta_rag['answer']}")



PERGUNTA: Quanto vocês cobram pelo serviço?
RESPOSTA: Nossa taxa de administração é de 8% sobre o valor do aluguel. Essa taxa cobre a gestão completa do seu imóvel, incluindo divulgação, gestão de contratos, repasse garantido e suporte. Não há outros custos adicionais além disso.
